In [1]:
import pandas
from sklearn.svm import SVC

In [2]:
#1
data = pandas.read_csv('svm-data.csv', names = ['y', 'x1', 'x2'])
y = data['y']
X = data[['x1', 'x2']]

In [3]:
clf = SVC(C=10000, kernel='linear', random_state=241)
clf.fit(X, y)

SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=241,
    shrinking=True, tol=0.001, verbose=False)

In [6]:
with open('week3_1', 'w') as f:
    for i in clf.support_:
        if (i == clf.support_[-1]):
            f.write(str(i+1))
        else:
            f.write(str(i+1)+' ')

In [2]:
#2
import numpy as np
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [237]:
newgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

In [238]:
vectorizer = TfidfVectorizer()

In [239]:
X = vectorizer.fit_transform(newgroups.data)
y = newgroups.target

In [240]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = svm.SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=241,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': array([  1.00000000e-05,   1.00000000e-04,   1.00000000e-03,
         1.00000000e-02,   1.00000000e-01,   1.00000000e+00,
         1.00000000e+01,   1.00000000e+02,   1.00000000e+03,
         1.00000000e+04,   1.00000000e+05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [241]:
gs.cv_results_

{'mean_fit_time': array([ 2.89339085,  3.14146705,  3.17180233,  2.96373768,  2.35462308,
         1.44409008,  1.50693269,  1.48974361,  1.45045156,  1.41234512,
         1.41954041]),
 'std_fit_time': array([ 0.09429025,  0.24460848,  0.23859844,  0.17424846,  0.05293676,
         0.04012865,  0.07126104,  0.03862553,  0.04076665,  0.02560154,
         0.02382609]),
 'mean_score_time': array([ 0.6902915 ,  0.75363197,  0.76601   ,  0.69207683,  0.57974205,
         0.33614254,  0.35922904,  0.33664217,  0.33269186,  0.31971641,
         0.32103615]),
 'std_score_time': array([ 0.01783272,  0.05501029,  0.06062213,  0.02356935,  0.04172501,
         0.01130776,  0.01674682,  0.01902233,  0.01649601,  0.00870549,
         0.00897661]),
 'param_C': masked_array(data = [1.0000000000000001e-05 0.0001 0.001 0.01 0.10000000000000001 1.0 10.0
  100.0 1000.0 10000.0 100000.0],
              mask = [False False False False False False False False False False False],
        fill_value = ?),
 '

In [242]:
gs.best_params_['C']

1.0

In [243]:
clf = SVC(C = gs.best_params_['C'], kernel='linear', random_state=241)
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=241,
    shrinking=True, tol=0.001, verbose=False)

In [244]:
coef = clf.coef_

In [245]:
arr = abs(coef.toarray()[0])
arr.argmax()

24019

In [246]:
arr.argsort()[::-1][:10]

array([24019, 12871,  5088,  5093, 17802, 23673, 21850,  5776, 15606, 22936])

In [247]:
feature_mapping = vectorizer.get_feature_names()
top_words = []
for i in arr.argsort()[::-1][:10]:
    top_words.append(feature_mapping[i])
print(top_words)

['space', 'god', 'atheism', 'atheists', 'moon', 'sky', 'religion', 'bible', 'keith', 'sci']


In [248]:
with open('week3_2', 'w') as f:
    for word in sorted(top_words):
        if (word == sorted(top_words)[-1]):
            f.write(word)
        else:
            f.write(word + ' ')

In [3]:
from sklearn.metrics import roc_auc_score

In [5]:
#3
data = pandas.read_csv('data-logistic.csv', names = ['y', 'x1', 'x2'])
y = data['y']
X = data[['x1', 'x2']]

In [81]:
def new_w(X, y, w, k, t):
    n = len(w)
    l = len(y)
    w1 = np.zeros(n) 
    for i in range(n):
        S = 0
        for j in range(l):
            S += y.iloc[j]*X.iloc[j,i]*(1-1./(1+np.exp(-y[j]*(w[0]*X.iloc[j,0]+w[1]*X.iloc[j,1]))))
        w1[i] += w[i] + k/l*S - k*t*w[i]
    return w1

In [82]:
def loss_Q(X, y, w, t):
    l = len(y)
    Q = 0
    for i in range(l):
        Q += np.log2(1+np.exp(-y[i]*(w[0]*X.iloc[i,0]+w[1]*X.iloc[i,1])))
    Q = Q/l + t*sum([w[0]**2, w[1]**2])
    return Q

In [55]:
def dist_euclid(w1, w2):
    s = 0
    for i in range(len(w1)):
        s += (w1[i]-w2[i])**2
    return np.sqrt(s)

In [97]:
def logistic_regression(X, y, w, k, t, max_iter, eps):
    i = 0
    while (True):
        Q = loss_Q(X,y,w,t)
        w1 = new_w(X,y,w,k,t)
        if (dist_euclid(w,w1) < eps):
            break
        w = w1
        i += 1
        if (i == iter):
            print("Iter max")
            break
    y_scores = np.zeros(len(y))
    for i in range(len(y)):
        y_scores[i] += 1./(1+np.exp(-w1[0]*X.iloc[i,0]-w1[1]*X.iloc[i,1]))
    return y_scores

In [98]:
# без регуляризации
w = np.zeros(2)
t = 0
k = 0.1
max_iter = 10000
eps = 0.00001

y_scores = logistic_regression(X,y,w,k,t,max_iter,eps)
roc = roc_auc_score(y, y_scores)

In [99]:
# с регуляризацией
w_regular = np.zeros(2)
t = 10
k = 0.1
max_iter = 10000
eps = 0.00001

y_scores_regular = logistic_regression(X,y,w_regular,k,t,max_iter,eps)
roc_regular = roc_auc_score(y, y_scores_regular)

In [100]:
with open('week3_3', 'w') as f:
    f.write(str(round(roc,3)) + ' ' + str(round(roc_regular,3)))
roc, roc_regular

(0.92685714285714282, 0.93628571428571417)

In [50]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [47]:
#4 
data = pandas.read_csv('classification.csv')

In [48]:
y_true = data['true']
y_pred = data['pred']

In [67]:
def true_false (y_true, y_pred):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i in range(len(y_true)):
        if (y_true[i] == 1):
            if (y_pred[i] == 1):
                TP += 1
            else:
                FN += 1
        else:
            if (y_pred[i] == 0):
                TN += 1
            else:
                FP += 1
    return (TP, TN, FP, FN)

In [68]:
def accuracy (TP, TN, FP, FN):
    return ((TP+TN)/(TP+TN+FP+FN))

def precision (TP, FP):
    return (TP/(TP+FP))

def recall (TP, FN):
    return (TP/(TP+FN))

def f_score (TP, FP, FN):
    prec = precision(TP, FP)
    rec = recall(TP, FN)
    return (2*prec*rec/(prec+rec))

In [69]:
TP, TN, FP, FN = true_false(y_true, y_pred)
acc = round(accuracy(TP, TN, FP, FN),2)
prec = round(precision(TP, FP),2)
rec = round(recall(TP, FN),2)
f_ = round(f_score(TP, FP, FN),2)

In [70]:
with open('week3_4', 'w') as f:
    f.write(str(TP) + ' ' + str(FP) + ' ' + str(FN) + ' ' + str(TN))
TP, FP, FN, TN

(43, 34, 59, 64)

In [71]:
with open('week3_5', 'w') as f:
    f.write(str(acc) + ' ' + str(prec) + ' ' + str(rec) + ' ' + str(f_))
acc, prec, rec, f_

(0.54, 0.56, 0.42, 0.48)

In [30]:
data = pandas.read_csv('scores.csv')

In [32]:
y_true = data['true']
score_logreg = data['score_logreg']
score_svm = data['score_svm']
score_knn = data['score_knn']
score_tree = data['score_tree']

In [35]:
roc_auc_score_logreg = roc_auc_score(y_true, score_logreg)
roc_auc_score_svm = roc_auc_score(y_true, score_svm)
roc_auc_score_knn = roc_auc_score(y_true, score_knn)
roc_auc_score_tree = roc_auc_score(y_true, score_tree)
print(round(roc_auc_score_logreg,2), round(roc_auc_score_svm,2), round(roc_auc_score_knn,2), round(roc_auc_score_tree,2))

0.72 0.71 0.64 0.69


In [36]:
with open('week3_6', 'w') as f:
    f.write('score_logreg')

In [37]:
from sklearn.metrics import precision_recall_curve

In [40]:
prec_logreg, rec_logreg, thresholds_logreg = precision_recall_curve(y_true, score_logreg)
prec_svm, rec_svm, thresholds_svm = precision_recall_curve(y_true, score_svm)
prec_knn, rec_knn, thresholds_knn = precision_recall_curve(y_true, score_knn)
prec_tree, rec_tree, thresholds_tree = precision_recall_curve(y_true, score_tree)

In [43]:
name = ""
max_prec = 0
m_rec = 0.7
for i in range(len(prec_logreg)):
    if (rec_logreg[i] >= m_rec and prec_logreg[i] > max_prec):
        max_prec = prec_logreg[i]
        name = 'score_logreg'
for i in range(len(rec_svm)):
    if (rec_svm[i] >= m_rec and prec_svm[i] > max_prec):
        max_prec = prec_svm[i]
        name = 'score_svm'
for i in range(len(rec_knn)):
    if (rec_knn[i] >= m_rec and prec_knn[i] > max_prec):
        max_prec = prec_knn[i]
        name = 'score_knn'
for i in range(len(rec_tree)):
    if (rec_tree[i] >= m_rec and prec_tree[i] > max_prec):
        max_prec = prec_tree[i]
        name = 'score_tree'
print(max_prec, name)

0.651785714286 score_tree


In [44]:
with open('week3_7', 'w') as f:
    f.write(name)